# 1. Build A LDA Model of 2012 London Olympics

## 1.1 Packages Importing

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet  import WordNetLemmatizer
from nltk import FreqDist
import pandas as pd
import numpy as py
import string
from gensim.models import CoherenceModel, LdaModel, LdaMulticore, Phrases, TfidfModel
from gensim.corpora import Dictionary
from gensim.models.phrases import Phraser
# libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 1.2 Parameters Setting 

In [4]:
col = 2                             # Define which columns are used for topic modelling.
coherence_type = 'c_v'       # Define coherence type
min_len = 4                       # Define the short word length
no_below =  2                   #a threshold filtering out number of occurrence of the tokens among documents  no larger than 5
no_above = 0.9                 #the portion of a word in total corpus size
chunksize = 2000              #Number of documents to be used in each training chunk
eval_every = None             # Don't evaluate model perplexity, takes too much time.
num_topics =  20              #The number of requested latent topics to be extracted from the training corpus.
num_passes = 20              #Number of passes through the corpus during training
iterations = 100                #Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.
alpha = 50.0/num_topics   #expresses our a-priori belief for each topics’ probability
#eta = 'auto'     
eta =      0.01                  #A-priori belief on word probability
random_state  = 54321     #Either a randomState object or a seed to generate one. Useful for reproducibility.
lemmatization = 1                  #Whether implement corpus lemmatization 2:Lemmatizer (without POS tags) 1:Lemmatizer (with POS tags) 0:no

## 1.3 Corpus Reading 

In [5]:
df = pd.read_csv("Olympics-ALL-DOCS.csv", header = None)

In [22]:
df

,0,1,2
0,1,Olympics-All-DOCS/NewChunk1/Examination_of_Wit...,Examination of Witnesses SEPTEMBER MS BAR...
1,2,Olympics-All-DOCS/NewChunk1/Examination_of_Wit...,Examination of Witnesses SEPTEMBER MS BAR...
2,3,Olympics-All-DOCS/NewChunk1/Examination_of_Wit...,Examination of Witnesses SEPTEMBER MS BAR...
3,4,Olympics-All-DOCS/NewChunk1/Jan_2003_-_Qs_1-19...,Select Committee on Culture Media and Sport M...
4,5,Olympics-All-DOCS/NewChunk1/Jan_2003_-_Qs_100-...,Select Committee on Culture Media and Sport M...
...,...,...,...
290,What had not happened at that stage was the d...,NaN,NaN
291,eached settlement anyway and we havent been so...,NaN,NaN
292,gret that Examination of Witnesses Witnesses...,NaN,NaN
293,will have been contacted by Join In Trust I ho...,NaN,NaN


## 1.4 Data Processing

In [ ]:
# function to plot most frequent terms
def freq_words(x, terms = 30):
    all_words = ' '.join(['%s' %text for text in x])
    all_words = all_words.split()
    
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
    
    # selecting top 20 most frequent words
    d = words_df.nlargest(columns="count", n = terms) 
    plt.figure(figsize=(20,5))
    ax = sns.barplot(data=d, x= "word", y = "count")
    ax.set(ylabel = 'Count')
    plt.show()

print('Word number: %d'% (len(df[col])))
freq_words(df[2],10) 

In [ ]:
#Tokenize the corpus 
tokenizer = RegexpTokenizer(r'\w+')
for i in range (len(df)):
    df[col][i] = df[col][i].lower()
    df[col][i] = tokenizer.tokenize(df[col][i])

print('Tokens number: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

In [ ]:
#Remove the number
df[col] = [ [token for token in doc if not token.isnumeric()] for doc in df[col]]

print('Tokens number after removing number: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

In [ ]:
#Reomve stop words
stop_words = stopwords.words('english')
df[col] = [ [token for token in doc if not token in stop_words] for doc in df[col] ]
print('Tokens number after removing stop words: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

In [ ]:
#Remove short words (length< min_len)
df[col] = [ [token for token in doc if not len(token) < min_len] for doc in df[col] ]
print('Tokens number after removing short words: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

In [ ]:
# Corpus Lemmatization
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemmatize_sentence(sentence):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(sentence):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return res

if lemmatization == 1:
    for i in range (len(df)):
        df[col][i] = lemmatize_sentence(df[col][i])
    print('Lemmatization with POS tags...')

if lemmatization == 2:
    lemmatizer = WordNetLemmatizer()
    df[col] = [ [lemmatizer.lemmatize(token) for token in doc ] for doc in df[col] ]
    print('Lemmatization without POS tags...')

if lemmatization == 0:
    print('Cancel lemmatization...')

print('Tokens number after lemmatization: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

In [ ]:
#Remove all punctuations
#punctuation_string = string.punctuation
#for j in range (len(df)):
#    for i in punctuation_string:
#        df[col][j] = str(df[col][j]).replace(i, '')

In [ ]:
# Make bigram
bigrams_phases = Phrases(df[col], min_count = 10)
for i in range(len(df[col])):
    for token in bigrams_phases[df[col][i]]:
        if '_' in token:
            df[col][i].append(token)            

print('Tokens number after adding bigram: %d'% (len(Dictionary(df[col]))))
freq_words(df[col],10) 

## 1.5 The LDA Model Building

In [ ]:
# Build a dictionary
LOdict = Dictionary(df[col])

print('Tokens number after creating dictionary: %d'% (len(LOdict)))

In [ ]:
#Remove  words occur less than no_below documents and more then no_above/documents
LOdict.filter_extremes(no_below = no_below, no_above = no_above)

print('Tokens number after tf-idf: %d'% (len(LOdict)))

In [ ]:
# Create bag of word for corpus
LOcorpus = [LOdict.doc2bow(doc) for doc in df[col]]

In [ ]:
#Mapping from word IDs to words. It is used to determine the vocabulary size, as well as for debugging and topic printing.
id2word = LOdict 

In [ ]:
# Buila a LDA model for LONDON OLYPLMICS
LOldamodel = LdaModel(
    corpus = LOcorpus,
    id2word = id2word,
    chunksize = chunksize,
    alpha = alpha,
    eta = eta,
    iterations = iterations,
    num_topics = num_topics,
    passes = num_passes,
    eval_every = eval_every,
    random_state = random_state
)

In [ ]:
#Compute Coherence Score
coherence_model_lda = CoherenceModel(model = LOldamodel, texts = df[col], dictionary = LOdict, coherence=coherence_type)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score ', coherence_type,'=', coherence_lda, ' with number of topics=', num_topics)

In [ ]:
tc_score = pd.DataFrame(pd.read_excel("topic coherence.xlsx"))
tc = sns.lineplot(x = 'Topics Number', y = 'Score', hue = 'Topics Coherence Type', data = tc_score)
tc.set(title = 'Topic Coherence Score')
plt.xticks([20,21,22,23,24,25,26,27,28,29,30])
#tc.set_xticks(range(len(tc_score))) # <--- set the ticks first
#tc.set_xticklabels(['20','21','22','23','24','25','26','27','28','29','30'])

## 1.6 Corpus Classification

In [ ]:
#Make a classifier to classify each paragraph of London Olympics Corpus
classifer = list()
for d in df[col]:
    bow = LOdict.doc2bow(d)
    belong = LOldamodel.get_document_topics(bow,per_word_topics=False)
    classifer.append(belong)
    
classiferdf = pd.DataFrame(classifer)

In [ ]:
#Transform London Olympics Corpus to Dataframe
LOdf = pd.DataFrame(df[col])

In [ ]:
#Make the Dataframe of the biggest distribution topics of each paragraph
MTopics =  list()
for i in LOldamodel.get_document_topics(LOcorpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    #print(i[bz][0],i,listj,listj.index(max(listj)))
    #print(i[bz][0])
    MTopics.append(bz)

MTopics = pd.DataFrame(MTopics)

In [ ]:
#Integrate dataframe of orgrinal corpus, classifier, and the biggest distribution of each paragraph
classifer_document = pd.concat([LOdf,MTopics,classiferdf],axis=1)
classifer_document

In [ ]:
#Output final dataframe
classifer_document.columns = ["Corpus","Biggest distribution topic","T0%","T1%","T2%","T3%","T4%","T5%","T6%","T7%","T8%","T9%","T10%",
                             "T11%","T12%","T13%","T14%","T15%","T16%","T17%","T18%","T19%"
                             ]

classifer_document.to_csv(path_or_buf = 'C:/Users/jason/Documents/Exeter University/business project/code/output/classifer.csv')

In [ ]:
#Output topics distribution of whole corpus
topic_distribution = pd.DataFrame(LOldamodel.print_topics(num_topics = num_topics))
topic_distribution.columns = ["Topic index","Topic Distribution"]
topic_distribution
topic_distribution.to_csv(path_or_buf = 'C:/Users/jason/Documents/Exeter University/business project/code/output/topic distribution.csv',index=False)

## 2. The LDA Model Visulization

### 2.1 Packages Importing

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensim_models

### 2.2 Visulization

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(LOldamodel, LOcorpus, id2word)
vis